In [1]:
import numpy as np
import pandas as pd

import os
import flask
import json

from flask import Flask, session, redirect, url_for, render_template, request
from werkzeug.serving import run_simple

In [2]:
# Connect to the database and setup the app
app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

# Load data

In [3]:
df = pd.read_csv("df_pred_ext (1).csv").drop(["Unnamed: 0", 
                                            "job_order", 
                                            "source", 
                                            "time_between"], axis=1)

In [4]:
df.head()

,candidate_id,time_spent,isco_functie_niveau,education,company_name,function_id,isco_code4
0,84556,156,2.0,0.0,324258,936,208
1,84556,116,1.0,0.0,324258,809,348
2,84556,275,2.0,0.0,329244,936,208
3,84556,4,1.0,0.0,368140,1519,344
4,84556,11,1.0,0.0,419826,1519,344


In [5]:
certs = pd.read_csv("data/candidate_certificates_mappings.csv").drop("Unnamed: 0", axis=1)
certs = certs.groupby("candidate_id")["candidate_certificate_id"].apply(lambda x: x.values)

In [6]:
isco_codes = pd.read_csv("data/isco translations it sp uk nl fr de - Sheet1.csv")[["ISCO_08_Code", 
                                                                                   "NL ISCO "]].set_index(
    "ISCO_08_Code").to_dict()["NL ISCO "]

# isco_codes = {int(k): v for k, v in isco_codes.items() if k.isdigit()}

In [7]:
languages = pd.read_csv("data/languages_mappings.csv").drop("Unnamed: 0", axis=1)
languages = languages.groupby("candidate_id")["language_id"].apply(lambda x: x.values)

In [8]:
licenses = pd.read_csv("data/license_mappings.csv")
licenses = licenses.groupby("candidate_id")["driving_licenses"].apply(lambda x: x.values)

In [9]:
addresses = pd.read_csv("data/address_mappings.csv").set_index("candidate_id").to_dict()["from_post_code"]

In [10]:
skills = pd.read_csv("data/candidate_skills_mapped.csv").drop(["Unnamed: 0", "date_start"], axis=1)
skills = skills.groupby("candidate_id")["skill_id"].apply(lambda x: x.values)

In [11]:
with open("data/index_to_company.json") as f:
    companies = json.load(f)
    companies = {int(k): v for k, v in companies.items()}

In [12]:
with open("data/index_to_isco.json") as f:
    isco = json.load(f)
    isco = {int(k): int(v) for k, v in isco.items()}

In [13]:
skill_levels = {1 : "Routineus werk",
                2 : "Gemiddeld werk",
                3 : "Bovengemiddeld werk",
                4 : "Kenniswerk"}

# Convert DataFrame values to human-readable text

In [14]:
df["company_name"] = df["company_name"].apply(lambda x: companies[x] if x in companies else x)

In [15]:
df["isco_code4"] = df["isco_code4"].apply(lambda x: isco[int(x)] if int(x) in isco else int(x))

In [16]:
df["isco_code4"] = df["isco_code4"].apply(lambda x: isco_codes[x])

In [17]:
df["location"] = df["candidate_id"].apply(lambda x: addresses[x] if x in addresses else "N.v.t.")

In [18]:
df["skills"] = df["candidate_id"].apply(lambda x: ", ".join(skills[x]) if x in skills else "N.v.t.")

In [19]:
df["certificates"] = df["candidate_id"].apply(lambda x: ", ".join(certs[x]) if x in certs else "N.v.t.")

In [20]:
df["licenses"] = df["candidate_id"].apply(lambda x: ", ".join(licenses[x]) if x in licenses else "N.v.t.")

In [21]:
df["languages"] = df["candidate_id"].apply(lambda x: ", ".join(languages[x]) if x in languages else "N.v.t.")

In [22]:
df["isco_functie_niveau"] = df["isco_functie_niveau"].apply(lambda x: skill_levels[int(x)])

In [23]:
df.head()

,candidate_id,time_spent,isco_functie_niveau,education,company_name,function_id,isco_code4,location,skills,certificates,licenses,languages
0,84556,156,Gemiddeld werk,0.0,"Rest. Les 3 suisses, CH",936,Koks,1102,"Klantgerichtheid, Aanpassingsvermogen, Communi...",N.v.t.,B,Nederlands
1,84556,116,Routineus werk,0.0,"Rest. Les 3 suisses, CH",809,Bereiders van fastfood,1102,"Klantgerichtheid, Aanpassingsvermogen, Communi...",N.v.t.,B,Nederlands
2,84556,275,Gemiddeld werk,0.0,Logidex BV,936,Koks,1102,"Klantgerichtheid, Aanpassingsvermogen, Communi...",N.v.t.,B,Nederlands
3,84556,4,Routineus werk,0.0,Little indian Horses,1519,Inpakkers,1102,"Klantgerichtheid, Aanpassingsvermogen, Communi...",N.v.t.,B,Nederlands
4,84556,11,Routineus werk,0.0,Tassos island,1519,Inpakkers,1102,"Klantgerichtheid, Aanpassingsvermogen, Communi...",N.v.t.,B,Nederlands


In [24]:
df_grouped = df.groupby("candidate_id")

# Flask 

In [25]:
@app.route('/', methods=["GET", "POST"])
def index():
    if request.method == "POST":
        session["user_type"] = request.form['submit_button']
        session["example"] = 0
        return redirect(url_for("sliders"))
    else:
        return render_template("/index.html")

In [26]:
@app.route('/sliders', methods=["GET", "POST"])
def sliders():
    
    if session["example"] >= 3:
        return "klaar!"
    
    # instead --> get examples[group_type][example_number]    
    df = df_grouped.get_group(np.random.choice(
        list(df_grouped.indices.keys()))
                             ).reset_index().drop(["index", 
                                                   "candidate_id"], axis=1)
        
    static_features = pd.DataFrame(df[["location", "skills", "certificates", "licenses", "languages"]].iloc[0].T)
    df = df.drop(["location", "skills", "certificates", "licenses", "languages"], axis=1)
    
    df.index.name = "Baan nummer"
    df.index += 1
    df = df.T
    df.index = ["Dagen gewerkt", "Werkniveau", "Opleidingsniveau", "Bedrijf", "Functie", "Isco code"]
    df = df.loc[["Isco code", "Functie", "Bedrijf", "Opleidingsniveau", "Dagen gewerkt", "Werkniveau"]]

    static_features.index = ["Postcode", "Vaardigheden", "Certificaten", "Rijbewijzen", "Talen"]
    static_feautres = static_features.loc[["Vaardigheden", "Certificaten", "Talen", "Rijbewijzen", "Postcode"]]
    
    return render_template("/sliders.html", 
                           dataframe=df.to_html(classes="table"), 
                           static_features=static_features.to_html(header=False, classes=["table", "static_data"]))

In [27]:
@app.route("/store_results", methods=["POST"])
def store_results():
    
    r = request.get_json()
    values = dict(zip(["isco code", "function id", "company", "education", "days worked", 
                       "isco level", "CV", "skills", "certificates", "languages", "licenses",
                       "location"], r["values"]))
    
    print(session["user_type"], session["example"], values)
    session["example"] += 1
        
    return redirect(url_for("sliders"))

In [ ]:
run_simple('localhost', 8080, app)

 * Running on http://localhost:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [23/May/2022 15:37:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 15:37:59] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [23/May/2022 15:37:59] "GET /sliders HTTP/1.1" 200 -
